In [ ]:
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import glob

file_list = glob.glob("./fits_files/Run*.fits.gz")
file_list.sort()
#print(file_list)

decs = []
sigs = []
enrgs = []
fars = []
decs_g = []
sigs_g = []
enrgs_g = []
fars_g = []
for fitsfile in file_list:
    skymap, header = hp.read_map(fitsfile,h=True, verbose=False)
    header = dict(header)
    if header is not None:
        class_string = ''
        for val in ['CASCADE_SCR','THRGOING_SCR','START_SCR','STOP_SCR','SKIMMING_SCR']:
            if header[val]>0.03:
                class_string = class_string + val + ':' + f'{header[val]:5} '
        print(f'{fitsfile[13:32]} MJD: {header["EVENTMJD"]:5.2f} SCAN RA: {header["RA"]:6} (+{header["RA_ERR_PLUS"]:4} -{header["RA_ERR_MINUS"]:4})' \
              f' DEC: {header["DEC"]:6} (+{header["DEC_ERR_PLUS"]:4} -{header["DEC_ERR_MINUS"]:4})' \
              f' E: {header["ENERGY"]:7.2f} S: {(header["SIGNAL"]*100.0):5.2f}%  FAR: {header["FAR"]:7} type:{header["I3TYPE"]:10}' \
              f' {class_string}')
        if 'GCN_URL' in header:
            print(header['GCN_URL'])
        if 'GCN' in header:
            print('WRONG GCN NAME TAG!')
              #f' CSC_P: {header["CASCADE_SCR"]:5.3} THRU_P: {header["THRGOING_SCR"]:5.3} START_P: {header["START_SCR"]:5.3} STOP_P: {header["STOP_SCR"]:5.3} SKIM_P:{header["SKIMMING_SCR"]:5.3}')
              #decs.append(header["DEC"])
        #sigs.append(header["SIGNAL"])
        #enrgs.append(header["ENERGY"])
        #fars.append(header["FAR"])
        if 'gold' in header["I3TYPE"]:
            decs_g.append(header["DEC"])
            sigs_g.append(header["SIGNAL"])
            enrgs_g.append(header["ENERGY"])
            fars_g.append(header["FAR"])
        else:
            decs.append(header["DEC"])
            sigs.append(header["SIGNAL"])
            enrgs.append(header["ENERGY"])
            fars.append(header["FAR"])
#print(decs,decs_g)
#print(sigs,sigs_g)
#print(enrgs,enrgs_g)
#print(fars,fars_g)


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
sindecs = np.sin(np.deg2rad(decs))
sindecs_g = np.sin(np.deg2rad(decs_g))

plt.hist([sindecs,sindecs_g],stacked=True, bins=16, label=['Bronze','Gold'])
plt.legend(loc="upper right")
plt.xlabel('Sin(dec)')
plt.ylabel('Number')
plt.title('Declination of 9yr catalog')
fig.savefig('sindec_dist.png', bbox_inches='tight')

print(len(sindecs), len(sindecs_g),len(sindecs) + len(sindecs_g) )

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.hist([sigs,sigs_g], bins=20,stacked=True,range=[0.,1.0],label=['Bronze','Gold'])
plt.legend(loc="upper right")
plt.xlabel('Signalness')
plt.ylabel('Number')
plt.title('Signalness of 9yr catalog')
fig.savefig('signalness_dist.png', bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
logenrgs = np.log10(enrgs)
logenrgs_g = np.log10(enrgs_g)

plt.hist([logenrgs,logenrgs_g], bins=20,stacked=True,label=['Bronze','Gold'])
plt.xlabel('Log10E (TeV)')
plt.ylabel('Number')
plt.title('Energy of 9yr catalog')
fig.savefig('energy_dist.png', bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.hist([fars,fars_g], bins=20,stacked=True,label=['Bronze','Gold'])
plt.xlabel('FAR (1/yr)')
plt.ylabel('Number')
plt.title('False alarm rate of 9yr catalog')
fig.savefig('far_dist.png', bbox_inches='tight')

In [ ]:
x = range(100)
y = range(100,200)
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(logenrgs,sindecs, s=10, c='b', marker="s", label='Bronze')
ax1.scatter(logenrgs_g,sindecs_g, s=10, c='r', marker="o", label='Gold')
plt.xlabel('Log10E (TeV)')
plt.ylabel('Sin(dec)')
plt.legend(loc='upper left')
fig.savefig('energ-dec_scatter.png', bbox_inches='tight')
plt.show()


#plt.scatter(logenrgs,sindecs, label='Bronze')
#plt.xlabel('Log10E (TeV)')
#plt.ylabel('Sin(dec)')
#plt.legend(loc='upper right', fontsize=16,)

#plt.show()